In [3]:
from scipy.linalg import eigh, eigvalsh
from scipy.sparse import coo_matrix
import numpy as np
import networkx,math,sys
import time
from scipy.stats import unitary_group, ortho_group
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

def psdrF(mat,r,d=None):
    if d is None:
        d = len(mat)
    vas,ves = eigh(mat)
    vas[vas<0]=0
    pvas=np.diag(vas*np.array([0]*(d-r)+[1]*r))
    ves = np.matrix(ves)
    return np.matmul(np.matmul(ves,pvas),ves.H)

def normF(mat,adj,d=None):
    if d is None:
        d = len(mat)
    return np.multiply(mat,adj) + 1*np.identity(d)

def ranH(d):
    Mr = np.random.rand(d,d)
    Mi = np.random.rand(d,d)
    M = (Mr+Mr.T)/2 + 1j*(Mi-Mi.T)/2
    return np.matrix(M)

def dimQ(ghf,r,t): # k is the rank of the projectors; r is the rank of the matrix, i.e., the dimension of the vectors; t is the number of steps in the see-saw method in each monitored period
    if isinstance(ghf,str):gh = networkx.from_graph6_bytes(ghf.encode())
    elif isinstance(ghf,list) or isinstance(ghf,np.matrix):gh = networkx.Graph(ghf)
    else:return('Accepted types:str of graph6 format,list of edges, np.matrix of adjacency matrix')
    d = len(gh)
    adjc = 1-np.matrix(networkx.adjacency_matrix(gh).A)-np.identity(d)
    iterM = ranH(d)
    tmp0 = d
    tmp1 = 2*d
    start0 = time.time()
    start = start0
    while abs(tmp1-tmp0)>1e-9 and start-start0<30:
        newtime = time.time()
        if newtime-start>3:
            iterM = iterM + ranH(d)/100
            start = newtime
        for i in range(t): # t is the number of steps in the see-saw method in each monitored period
            iterM = psdrF(iterM,r,d)
            iterM = normF(iterM,adjc,d)
        vas = eigvalsh(iterM,eigvals=(0,d-r-1))
        tmp0 = tmp1
        tmp1 = sum(abs(vas))
    #return [abs(tmp1-tmp0)<1e-9,tmp1<1e-9]
    if tmp1<1e-9:
        return 'True'
    else:
        return 'Undetermied'

def dimQM(ghf,r,lt): # lt is the number of rounds
    res = []
    for i in range(lt):
        tmp = dimQ(ghf,r,20)
        if tmp[0]:res.append(tmp[1])
        if tmp[1]<1e-6:return [len(res),tmp]
        if len(res)>=5:return [len(res),min(res)]
    return [len(res),min(res+[1000])]



In [2]:
from newdim import *

gh = [[0,1],[1,2],[2,3],[3,4],[0,4]] #edge format
gh = 'Dhc' #graph6 format
res = dimQ(gh,3,10) # graph, orthogonal rank to check, length of iteration in each small step (the larger the more accurate, then more time)
# The result 'Ture' means that we have a orthogonal representation with such a rank, 'Undetermined' means that it cannot be decided by this routine. We can always increase the targeted orthogonal rank to obtain the result 'True'.
print(res)

True


In [3]:
import csv
from csv import reader
from time import time

In [9]:
t0=time()
mini=1000

for vertices in range(18,19):
    mini=1000
    with open('vtc'+str(vertices)+'.ata','r') as read_obj:
        csv_reader = reader(read_obj,delimiter='\t')
        # header = next(csv_reader)
        l=0
        # Check file as empty
        # if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader: 
            # row variable is a list that represents a row in csv
            l=l+1
            lin=[row[0]]
            for s in row[1:]:
                n=eval(s)
                lin=lin+[n]
            # print(lin)
            d=int(vertices/lin[2])
            #print(d)
            res = dimQ(row[0],d,10)
            while res != 'True':
                d=d+1
                res = dimQ(row[0],d,10)
            eta=2*lin[1]/(lin[1]+(vertices/d))
            if eta <= mini:
                mini=min(mini,eta)
                print(l,' ',lin,' ',eta,' ',d)
    if mini > 1:
        mini=1
    print(vertices,'\t',mini,'\t',l)
                # row variable is a list that represents a row in csv

t1=time()

1   ['QsXP?_?????@?D?I_i?T??T?Ag?', 8, 8.72017916, 9]   40/29   5
3   ['Qs`Ha_hOHAA@AC?a?a?CH@?s?F_', 6, 7.6191111, 9]   1   3
5   ['Qsa@yGgI?gH@DDO_CGgbaOIg@x?', 6, 7.14002653, 9]   1   3
7   ['QsaBb`CQ@GbCI?D?_ioTQ?d_@hG', 6, 7.10875519, 9]   1   3
11   ['QsaCBp[[bSF_O`DA`aPDB?xWDYO', 6, 7.10875519, 9]   1   3
17   ['Q}udJ@`E?c`a@bCM@I_IL?io@`w', 4, 4.5, 4.5]   0.9411764705882353   4
34   ['Q}udK@`WWe`kCxBUGoXoWDI[IUo', 4, 4.5, 4.5]   0.9411764705882353   4


AttributeError: 'float' object has no attribute 'n'

In [10]:
print(vertices,'\t',mini,'\t','\t',l)

18 	 0.9411764705882353 	 	 149


In [12]:
t0=time()
mini=1000

for vertices in range(18,19):
    mini=1000
    with open('trans'+str(vertices)+'c.at','r') as read_obj:
        csv_reader = reader(read_obj,delimiter='\t')
        # header = next(csv_reader)
        l=0
        # Check file as empty
        # if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader: 
            # row variable is a list that represents a row in csv
            l=l+1
            lin=[row[0]]
            for s in row[1:]:
                n=eval(s)
                lin=lin+[n]
            # print(lin)
            d=int(vertices/lin[2])
            #print(d)
            res = dimQ(row[0],d,10)
            while res != 'True':
                d=d+1
                res = dimQ(row[0],d,10)
            eta=2*lin[1]/(lin[1]+(vertices/d))
            if eta <= mini:
                mini=min(mini,eta)
                print(l,' ',lin,' ',eta,' ',d)
    if mini > 1:
        mini=1
    print(vertices,'\t',mini,'\t',l)
                # row variable is a list that represents a row in csv

t1=time()

1   ['QqGG_CC?GA?@?C??_?_?@??O??W', 9, 9]   6/5   3
2   ['QsOP?`?G?[?K?S?g?a?O???s?Co', 9, 9]   1.0   2
3   ['QsP@@?OC?Q?oAC@G?Q?B??@c?H_', 9, 9]   1.0   2
7   ['Qs_qR@OA?OC?C?@s?d_C[?U_?u?', 9, 9]   1.0   2
8   ['Qs`Ha_gO@?@?C?@`_LOBg?f?CY?', 9, 9]   1.0   2
11   ['Qs`a`OWO@?A?A??u?e_EK@I_?]?', 9, 9]   1.0   2
12   ['Qs`jaOoA?_?????F_TOIg?J_@U?', 9, 9]   1.0   2
13   ['Qs`ra?_C?O@?A?@B_PoBo?\\?@q?', 9, 9]   1.0   2
19   ['Qsa@yGgI?gH@DDO_CGgbaOIg@x?', 6, 7.14002653]   1   3
20   ['QsaBZhKLAOE?A?C??IwDU?Fo?n?', 9, 9]   1.0   2
22   ['QsaBbPSM?WA_Q?H??\\_Am?MoAN?', 9, 9]   1.0   2
23   ['QsaBb`CQ@GbCI?D?_ioTQ?d_@hG', 6, 7.10875519]   1   3
24   ['QsaBb`KJAGD?E?D??JgD[?eo@f?', 9, 9]   1.0   2
25   ['QsaBb`KR@OC_E?D??RgD[?To@j?', 9, 9]   1.0   2
26   ['QsaBrgKB?WK?S?K??^?@m?Jo?^?', 9, 9]   1.0   2
27   ['Q{a?wxOQAGECICHA?V?Bo?`w@FO', 6, 6]   1.0   3
28   ['Q{aAHPHHa_E@ECDG@AgHS?pcBH_', 6, 6]   1.0   3
29   ['Q{aAaPCK`QCcCICP@OoKQAKSDQ_', 6, 6]   1.0   3
30   ['Q{aApoKDAQEPP?H?_MGDc?DS?ig'

ValueError: Requested eigenvalue indices are not valid. Valid range is [0, 17] and start <= end, but start=0, end=-1 is given

In [13]:
t0=time()
mini=1000

for vertices in range(18,32):
    mini=1000
    with open('grafosconminimo'+str(vertices)+'.tsv','r') as read_obj:
        csv_reader = reader(read_obj,delimiter='\t')
        # header = next(csv_reader)
        l=0
        # Check file as empty
        # if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader: 
            # row variable is a list that represents a row in csv
            l=l+1
            lin=[row[0]]
            for s in row[1:]:
                n=eval(s)
                lin=lin+[n]
            # print(lin)
            d=int(vertices/lin[2])
            #print(d)
            res = dimQ(row[0],d,10)
            while res != 'True':
                d=d+1
                res = dimQ(row[0],d,10)
            eta=2*lin[1]/(lin[1]+(vertices/d))
            if eta < mini:
                mini=min(mini,eta)
                #print(lin,' ',eta,' ',d)
    if mini > 1:
        mini=1
    print(vertices,'\t',mini,'\t','\t',l)

t1=time()

18 	 1 	 	 36
19 	 1 	 	 8
20 	 1 	 	 30
21 	 1 	 	 8
22 	 1 	 	 4
23 	 1 	 	 2
24 	 1 	 	 12
25 	 1 	 	 6
26 	 1 	 	 1
27 	 1 	 	 12
28 	 1 	 	 18
29 	 1 	 	 4
30 	 1 	 	 10
31 	 1 	 	 2


In [15]:
t0=time()
mini=1000

for vertices in range(33,48):
    mini=1000
    with open('grafosconminimo'+str(vertices)+'.table.tsv','r') as read_obj:
        csv_reader = reader(read_obj,delimiter='\t')
        # header = next(csv_reader)
        l=0
        # Check file as empty
        # if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader: 
            # row variable is a list that represents a row in csv
            l=l+1
            lin=[row[0]]
            for s in row[1:]:
                n=eval(s)
                lin=lin+[n]
            # print(lin)
            d=lin[3]
            #print(d)
            res = dimQ(row[0],d,10)
            while res != 'True':
                d=d+1
                res = dimQ(row[0],d,10)
            eta=2*lin[1]/(lin[1]+(vertices/d))
            if eta < mini:
                mini=min(mini,eta)
                #print(lin,' ',eta,' ',d)
    if mini > 1:
        mini=1
    print(vertices,'\t',mini,'\t','\t',l)

t1=time()

33 	 1 	 	 4
34 	 1 	 	 71
35 	 1 	 	 16
36 	 1 	 	 60
37 	 1 	 	 16
38 	 1 	 	 6
39 	 1 	 	 24
40 	 1 	 	 8
41 	 1 	 	 2


LinAlgError: Internal Error.

In [16]:
t0=time()
mini=1000

for vertices in range(42,48):
    mini=1000
    with open('grafosconminimo'+str(vertices)+'.table.tsv','r') as read_obj:
        csv_reader = reader(read_obj,delimiter='\t')
        # header = next(csv_reader)
        l=0
        # Check file as empty
        # if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader: 
            # row variable is a list that represents a row in csv
            l=l+1
            lin=[row[0]]
            for s in row[1:]:
                n=eval(s)
                lin=lin+[n]
            # print(lin)
            d=lin[3]
            #print(d)
            res = dimQ(row[0],d,10)
            while res != 'True':
                d=d+1
                res = dimQ(row[0],d,10)
            eta=2*lin[1]/(lin[1]+(vertices/d))
            if eta < mini:
                mini=min(mini,eta)
                print(lin,' ',eta,' ',d)
    if mini > 1:
        mini=1
    print(vertices,'\t',mini,'\t',l)

t1=time()

42 	 1 	 	 42


KeyboardInterrupt: 

In [17]:
t0=time()
mini=1000

for vertices in range(44,48):
    mini=1000
    with open('grafosconminimo'+str(vertices)+'.table.tsv','r') as read_obj:
        csv_reader = reader(read_obj,delimiter='\t')
        # header = next(csv_reader)
        l=0
        # Check file as empty
        # if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader: 
            # row variable is a list that represents a row in csv
            l=l+1
            lin=[row[0]]
            for s in row[1:]:
                n=eval(s)
                lin=lin+[n]
            # print(lin)
            d=lin[3]
            #print(d)
            res = dimQ(row[0],d,10)
            while res != 'True':
                d=d+1
                res = dimQ(row[0],d,10)
            eta=2*lin[1]/(lin[1]+(vertices/d))
            if eta < mini:
                mini=min(mini,eta)
                print(l,' ',lin,' ',eta,' ',d)
    if mini > 1:
        mini=1
    print(vertices,'\t',mini,'\t',l)

t1=time()

['ksaCCEPbCIWGcfiJTcUcbCWCQa?{?GWaKGH_KPAa_GhcOdIQE?nGM?bgOR]I@[[@HpBRdEG`|`BaFAGQwxA_DMh`AHIoR_QFaGx@DaLrG?XDJgiC^OBHD@nP?LDQaoYg_VOMWK?t[@ReRc?Da[eo_dOPld?Ceo_', 6, 9.7070731, 3, 0.5806451612903226]   102/73   17
['ksaCCECcSPx_k`qKuHlDgpQco?qOAX?c`?KGHSGQDQ@aQJaBAMaZc@XMp?[OHat@?qNOO\\QFGgtWLa_]IJUgUabsj]W@eEXVCH|KIk_ZwoN@[Je?U_yTe?KeqPPOJE[WggCKskiWD@KlLP_WHRxFG_BabuJCOD_', 6, 9.4242004, 3, 0.5806451612903226]   48/35   16
['k{fLn~}s\\RtNs^~a~qn{N~uN}x]~ld~]f~jL~|JeyzDVV[IxTePwjdaxygb|lTP]Cv}fkK~tlrQZ}vR`f~Mh\\@~}jMg^~TqMd|t{ctjmnkHrwi~XPlwT~e`jp^Sjwd{Nit^[UpFx\\|g{gnYzxuSqNmzyxgkVv\\{', 3, 4.6688913, 6, 0.5806451612903226]   78/61   26
44 	 1 	 4


KeyboardInterrupt: 

In [18]:
t0=time()
mini=1000

for vertices in range(45,48):
    mini=1000
    with open('grafosconminimo'+str(vertices)+'.table.tsv','r') as read_obj:
        csv_reader = reader(read_obj,delimiter='\t')
        # header = next(csv_reader)
        l=0
        # Check file as empty
        # if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader: 
            # row variable is a list that represents a row in csv
            l=l+1
            lin=[row[0]]
            for s in row[1:]:
                n=eval(s)
                lin=lin+[n]
            # print(lin)
            d=lin[3]
            #print(d)
            res = dimQ(row[0],d,10)
            while res != 'True':
                d=d+1
                res = dimQ(row[0],d,10)
            eta=2*lin[1]/(lin[1]+(vertices/d))
            if eta <= mini:
                mini=min(mini,eta)
                print(l,' ',lin,' ',eta,' ',d)
    if mini > 1:
        mini=1
    print(vertices,'\t',mini,'\t',l)

t1=time()

1   ['lsaCCECcSPx_k`qKuHlDgpQdG@H_@`?c`?KGGSHQ@QDAIO|AJ@{Hk@wgx?{ir?IyMW?mgQPYPOWPwPO{DIpW]DAxR@KVGkGEPuIQLwOUoc\\h@R_pU_yS@eF_mD?rBHkgSCQWrad@CHPzB_AGxRxJ?Gapmd_UoC@MlGSwC@', 6, 9.6999755, 3, 0.5714285714285714]   24/17   18
3   ['lv~fNflv|~]ayb|F|R~~h~yq^|S~t\\~xf^zKrjW\\mfBx{vYlxvJ}Lia^HyaNZXgb{{qa[cnnyyEnnyqwynncyYnnkRmF~sxmo~|ZENjtVuQ]ntVqIZmQ~qiMzEnug\\HK~vlDopf~ZsbF{mjVyHNumh^xwjBvUv|YJ`|s|~', 3, 4.6392143, 6, 0.5714285714285714]   4/3   30
4   ['lv~fNflv|~]ayb|F|R~~h~yq^|S~t\\~xf^zKrjW\\mfBx{vYlxvJ}Lia^HyaNZXgb{{qa[cnnyyEnnyqwynncyYnnkRmF~sxmo~|ZENjtVuQ]ntVqIZmQ~qiMzEnug\\HK~vlDopf~ZsbF{mjVyHNumh^xwjBvUv|YJ`|s|~', 3, 4.6392143, 6, 0.5714285714285714]   4/3   30
45 	 1 	 4
1   ['msaCCIAoT@x_k`kLXH[XccrCGSO_SPCELCCeQ?KP_AP@ICkCIOxWW@`dq?jYkODzOt@CociALIGapj@CPqU`{_DiG]o?|@?DauOh?hFeDADw`MjSBx?vFgLO]OOXbgJWGEWRTKQXK?qjGpWoGtk_SIG_y[OgSP?hyeGAGRS^Pc?PGo', 6, 9.7070731, 3, 0.5624999999999999]   108/77   18
2   ['m{fT^~}k]RxNk^~d~p~~t~~VvVF\\\\F}Vn}tmezlKzmUyycRjipF|^LL|^XXv{roK~tBhF

In [4]:
t0=time()
mini=1000

for vertices in range(32,33):
    mini=1000
    with open('grafosconminimo'+str(vertices)+'377.table.tsv','r') as read_obj:
        csv_reader = reader(read_obj,delimiter='\t')
        # header = next(csv_reader)
        l=0
        # Check file as empty
        # if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader: 
            # row variable is a list that represents a row in csv
            l=l+1
            lin=[row[0]]
            for s in row[1:]:
                n=eval(s)
                lin=lin+[n]
            # print(lin)
            d=lin[3]
            #print(d)
            res = dimQ(row[0],d,10)
            while res != 'True':
                d=d+1
                res = dimQ(row[0],d,10)
            eta=2*lin[1]/(lin[1]+(vertices/d))
            if eta <= mini:
                mini=min(mini,eta)
                print(l,' ',lin,' ',eta,' ',d)
    if mini > 1:
        mini=1
    print(vertices,'\t',mini,'\t',l)

t1=time()

TypeError: 'module' object is not callable

In [5]:
print(l)

377


In [18]:
t0=time()
mini=1000

for vertices in range(33,48):
    mini=1000
    with open('grafosconminimo'+str(vertices)+'.table.tsv','r') as read_obj:
        csv_reader = reader(read_obj,delimiter='\t')
        # header = next(csv_reader)
        l=0
        # Check file as empty
        # if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader: 
            # row variable is a list that represents a row in csv
            l=l+1
            lin=[row[0]]
            for s in row[1:]:
                n=eval(s)
                lin=lin+[n]
            #print(lin)
            d=1+int(vertices/lin[2])
            #print(d)
            res = dimQ(row[0],d,10)
            while res != 'True':
                d=d+1
                res = dimQ(row[0],d,10)
            eta=2*lin[1]/(lin[1]+(vertices/d))
            if eta <= mini:
                mini=min(mini,eta)
                print(lin,' ',1+int(vertices/lin[2]),' ',d)
    if mini > 1:
        mini=1
    print(vertices,'\t',mini.n(),'\t',l)
                # row variable is a list that represents a row in csv

t1=time()

6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
['hsb~vrwtDSrYfJhtwttn@rwVy_Fy?NQoJowAKktyYX^ieVsIrVpLT]uHT^fHX}PKSvkPc`FJga@Lja?bkdCkHmLCMPSJkYxDQmRjeaVQV}QbVBVyJDW]bxIP[Na{OXpDL\\rCuPPtu[', 4, 9.6999755, 4, 0.5614035087719298]   5   21
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
41 	 1.00000000000000 	 2


In [20]:
t0=time()
mini=1000

for vertices in range(18,31):
    mini=1000
    with open('grafosconminimo'+str(vertices)+'.tsv','r') as read_obj:
        csv_reader = reader(read_obj,delimiter='\t')
        # header = next(csv_reader)
        l=0
        # Check file as empty
        # if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader: 
            # row variable is a list that represents a row in csv
            l=l+1
            lin=[row[0]]
            for s in row[1:]:
                n=eval(s)
                lin=lin+[n]
            # print(lin)
            d=1+int(vertices/lin[2])
            #print(d)
            res = dimQ(row[0],d,10)
            while res != 'True':
                d=d+1
                res = dimQ(row[0],d,10)
            eta=2*lin[1]/(lin[1]+(vertices/d))
            mini=min(mini,eta)
    if mini > 1:
        mini=1
    print(vertices,'\t',mini,'\t',mini.n(),'\t',l)
                # row variable is a list that represents a row in csv

t1=time()

18 	 1 	 1.00000000000000 	 36
19 	 1 	 1.00000000000000 	 8
20 	 1 	 1.00000000000000 	 30
21 	 1 	 1.00000000000000 	 8
22 	 1 	 1.00000000000000 	 4
23 	 1 	 1.00000000000000 	 2
24 	 1 	 1.00000000000000 	 12
25 	 1 	 1.00000000000000 	 6
26 	 1 	 1.00000000000000 	 1
27 	 1 	 1.00000000000000 	 12
28 	 1 	 1.00000000000000 	 18
29 	 1 	 1.00000000000000 	 4
30 	 1 	 1.00000000000000 	 10


In [5]:
t0=time()
mini=1000

for vertices in range(31,32):
    mini=1000
    with open('grafosconminimo'+str(vertices)+'.tsv','r') as read_obj:
        csv_reader = reader(read_obj,delimiter='\t')
        # header = next(csv_reader)
        l=0
        # Check file as empty
        # if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader: 
            # row variable is a list that represents a row in csv
            l=l+1
            lin=[row[0]]
            for s in row[1:]:
                n=eval(s)
                lin=lin+[n]
            # print(lin)
            d=1+int(vertices/lin[2])
            #print(d)
            res = dimQ(row[0],d,10)
            while res != 'True':
                d=d+1
                res = dimQ(row[0],d,10)
            eta=2*lin[1]/(lin[1]+(vertices/d))
            mini=min(mini,eta)
    if mini > 1:
        mini=1
    print(vertices,'\t',mini,'\t',mini.n(),'\t',l)
                # row variable is a list that represents a row in csv

t1=time()

31 	 1 	 1.00000000000000 	 2


In [ ]:
t0=time()
mini=1000

for vertices in range(32,33):
    mini=1000
    with open('grafosconminimo'+str(vertices)+'.table.tsv','r') as read_obj:
        csv_reader = reader(read_obj,delimiter='\t')
        # header = next(csv_reader)
        l=0
        # Check file as empty
        # if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader: 
            # row variable is a list that represents a row in csv
            l=l+1
            lin=[row[0]]
            for s in row[1:]:
                n=eval(s)
                lin=lin+[n]
            # print(lin)
            d=1+int(vertices/lin[2])
            #print(d)
            res = dimQ(row[0],d,10)
            while res != 'True':
                d=d+1
                res = dimQ(row[0],d,10)
            eta=2*lin[1]/(lin[1]+(vertices/d))
            if eta <= mini:
                print(lin,' ',mini)
                mini=min(mini,eta)
            if l%10==0:
                print(l)
    if mini > 1:
        mini=1
    print(vertices,'\t',mini,'\t',mini.n(),'\t',l)
                # row variable is a list that represents a row in csv

t1=time()

['_saKeIbqT`uYrJf}GIH@QQCgoKWY`?dOdDAlGhFh[@vioE}]_Jd^?FYAYii?XVT@RS`poqkJJJNL_@bo|_@c', 4, 5.9227311, 4, 0.6666666666666666]   1000
['_saSTJAk]RUdqtf}GDHAaKDDGIgi__TPDAamHPFYk@vdoE}^?JU]_FhAdjQ?UfJ@LT@ppIiJJJdl_@bY\\_@c', 4, 5.9139321, 4, 0.6666666666666666]   26/21
['_sadeUSaESpMl^tnGGgQhCgOqAIQPzo@{kO|JGFI?x@gbFRDgEzHpRChJ_nBx?KxLg_rlQxoH@JlWAJuT_AK', 4, 5.9303964, 4, 0.6666666666666666]   26/21
['_seCMbHgtHvKpff}GKH?qSOgaSXGWiPDSF_eEEEW{@~bKiq@eyX^_BqG{OsHowQb[sKKLkgoozcLwKBWLeKC', 4, 5.7495471, 4, 0.6666666666666666]   26/21
['_seFDETp[WotmRunK?I?dOB\\Y?pEGHQP{WpjEBdIF_}eKVQ@NPELwI@eosURWd`dZQJQpFKYDB{?kIGxcF@k', 4, 5.5969179, 4, 0.6666666666666666]   26/21
['_seS\\aciLEYeqrf}G?G{@STLAkyHTaQdcSs[AqczaDOkcSRWbHHYDGYahK[WTbWx^_Fo?`xO~@iWEFPTcWWs', 4, 5.7338, 4, 0.6666666666666666]   26/21
10
['_siCeERjSXYRhZv]GKGGUX_Ky?wf@WWo|OcdHCe`OSlj_E^SpAtOxJMIBkSPRaap^_?NokYLbJMl_@`t\\_@c', 4, 5.9418913, 4, 0.6666666666666666]   26/21
['_siCeeQbLJWVjTv]?bI@ICwC]?wWdXOpXw?zSHDyeGImoCVOxEhZ_FQd`

In [ ]:
t0=time()
mini=1000

for vertices in range(33,34):
    mini=1000
    with open('grafosconminimo'+str(vertices)+'.tsv','r') as read_obj:
        csv_reader = reader(read_obj,delimiter='\t')
        # header = next(csv_reader)
        l=0
        # Check file as empty
        # if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader: 
            # row variable is a list that represents a row in csv
            l=l+1
            lin=[row[0]]
            for s in row[1:]:
                n=eval(s)
                lin=lin+[n]
            # print(lin)
            d=1+int(vertices/lin[2])
            #print(d)
            res = dimQ(row[0],d,10)
            while res != 'True':
                d=d+1
                res = dimQ(row[0],d,10)
            eta=2*lin[1]/(lin[1]+(vertices/d))
            mini=min(mini,eta)
    if mini > 1:
        mini=1
    print(vertices,'\t',mini,'\t',mini.n(),'\t',l)
                # row variable is a list that represents a row in csv

t1=time()